In [1]:
import torch
from torch.autograd import grad
import numpy as np
Mass = 1000

coordinates = [1000.0,3*Mass,np.pi/2,0.0] #t,r,theta,phi

coords = torch.tensor(coordinates,requires_grad=True)



In [2]:
from GRPytorch_models import SpaceTimeMetricModule
from GRPytorch_metrics import schwarchild_metric

schwar_metric_function = lambda x: schwarchild_metric(x,Mass=Mass)
schwar_module = SpaceTimeMetricModule(schwar_metric_function)

In [3]:
from GRPytorch_models import ChristoffelSymbols

# create a christoffel module
christoffel = ChristoffelSymbols(schwar_module)

In [6]:
# evaluate on some coordinates
christoffel_eval = christoffel(torch.tensor([coordinates,coordinates],requires_grad=True))

In [7]:

# \Gamma^{1}_{i,j} for the first coordinate
christoffel_eval[0,1,:,:]

tensor([[ 3.7037e-05,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -3.3333e-04,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -1.0000e+03,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+03]],
       grad_fn=<SliceBackward0>)

In [4]:
# Compare with known analytic values
r = 3*Mass
# symbols taken from pdf source
(Mass/(r*r)) * (1-(2*Mass/r)),-(Mass/(r*r)) / (1-(2*Mass/r)), -r*(1-(2*Mass/r))

(3.703703703703704e-05, -0.0003333333333333333, -1000.0000000000001)

In [5]:
from GRPytorch_models import RiemannTensor

# Create riemann tensor module
riemann_tensor = RiemannTensor(schwar_module)

In [7]:
# evaluate for two coordinates
coord_batch = torch.tensor([coordinates,coordinates],requires_grad=True)

rieman_for_coord = riemann_tensor(coord_batch)

In [8]:
# R^{0}_{1,0,1} for the first coordinate in the coordinate batch
rieman_for_coord[0,0,1,0,1]


tensor(2.2222e-07, grad_fn=<SelectBackward0>)

In [10]:
from GRPytorch_models import RiemannTensor
from GRPytorch_models import RicciTensor

# create a ricci tensor module
ricci_tensor = RicciTensor(schwar_module)

In [11]:

# evaluate the ricci tensor on 1000 coordinate points at once
coord_batch_10 = torch.tensor([coordinates] * 1000,requires_grad=True)
ricci_val = ricci_tensor(coord_batch_10)

In [12]:
ricci_val[0,:,:]

tensor([[-1.7764e-15,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.8422e-14,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  5.9605e-08]],
       grad_fn=<SliceBackward0>)

In [25]:
rieman_for_coord[0,0,1,0,1] + rieman_for_coord[0,1,1,1,1] + rieman_for_coord[0,2,1,2,1] + rieman_for_coord[0,3,1,3,1]

tensor(2.8422e-14, grad_fn=<AddBackward0>)

In [13]:
from GRPytorch_models import EinsteinTensor

# Einstein module
eins_tensor = EinsteinTensor(schwar_module)

In [14]:

eins_val = eins_tensor(coord_batch)

In [15]:
# Display einstein tensor for first coordinate
eins_val[0,:,:]

tensor([[ 2.8984e-15,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -1.3651e-14,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -6.6613e-08,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -6.6613e-08]],
       grad_fn=<SliceBackward0>)

Note I have not figure out a way to remove the numerical round off error.